In [6]:
import numpy as np
import decimal
from collections import OrderedDict
import time
import pprint

"""
Test parameters:
Lambda = 1.55e-6
n_0 = 1.52
n_1 = 1.55
d = 1.5e-6
a = 1.5e-6
p_q = (5,5)
step_size = 1
Result in seconds:
1.864, 1.839, 1.873
"""



#Defining variables
Lambda = 1.55e-6                      #Wavelength of the light in m
P = 10                                #Total Power in watts
n_0 = 1.45                           #Cladding refractive index. With 1.53, 1.54 weird result is obtained (1.544)
n_1 = 1.6                            #Core refractive index
d = 4.0e-6                            #Half Thickness of waveguide in metres
a = 16.0e-6                            #Half width of waveguide in metres
p_q = (4,4)                           #Tuple that will search for modes p and q up to the specified numbers
step_size = 1                         #Step size for the iterations when finding the solution
k_x = 0                               #Transverse wavenumbers
k_y = 0                               #Transverse wavenumbers
gamma_x = 0                           #Transverse wavenumbers
gamma_y = 0                           #Transverse wavenumbers
beta = 0                              #Propagation constant
phi = 0                               #Optical phase
psi = 0                               #Optical phase
mu_0 = (4*np.pi)*1e-7                 #Permeability of free space
epsilon_0 = 8.854e-12                 #Permittivity of free space

#Start timing
start = time.time()

#Calculating preliminary parameters
k = 2*np.pi / Lambda                  #Wavenumber or circular wavenumber
w_ang = 2.998e8 * k
p_modes = np.resize(np.arange(1,p_q[0]+1,1),(p_q[0],1))
p_modes.resize(p_q[0],1)
q_modes = np.resize(np.arange(1,p_q[1]+1,1),(p_q[1],1))
q_modes.resize(p_q[1],1)

#Solution for E^x_pq
#Transverse wavenumbers calculation
solution_x = OrderedDict()
solution_y = OrderedDict()
range_vector = np.arange(0,int(np.sqrt(k**2*(n_1**2-n_0**2)))+1,step_size)
range_vector.resize(1,int(np.sqrt(k**2*(n_1**2-n_0**2)))+1)

#Wavenumbers in x
gamma_x = np.zeros((p_q[0],int(np.sqrt(k**2*(n_1**2-n_0**2)))+1), dtype = np.float64)
gamma_x = range_vector+gamma_x
k_x = np.sqrt(k**2*(n_1**2-n_0**2)-gamma_x**2)
upperpart = gamma_x * n_1**2
lowerpart = k_x * n_0**2

solution_x = OrderedDict()
for n in range(len(gamma_x)): 
    wz = np.absolute(n*np.pi*0.5+np.arctan(upperpart[n]/lowerpart[n])-k_x*a)
    if np.argmin(wz) != 0:
        solution_x[str(n+1)]={'gamma_x':gamma_x[n][np.argmin(wz)],'k_x':k_x[n][np.argmin(wz)]}

#Wavenumbers in y
gamma_y = np.zeros((p_q[1],int(np.sqrt(k**2*(n_1**2-n_0**2)))+1), dtype = np.float64)
gamma_y = range_vector+gamma_y
k_y = np.sqrt(k**2*(n_1**2-n_0**2)-gamma_y**2)

solution_y = OrderedDict()
for n in range(len(gamma_y)): 
    x = np.absolute(n*np.pi*0.5+np.arctan(gamma_y[n]/k_y[n])-k_y*d)
    if np.argmin(x) != 0:
        solution_y[str(n+1)]={'gamma_y':gamma_y[n][np.argmin(x)],'k_y':k_y[n][np.argmin(x)]}

#Combining the solutions:
solution_xy = OrderedDict()

for p,valx in solution_x.items():
    for q,valy in solution_y.items():
        beta = np.sqrt(k**2*n_1**2-(valx['k_x']**2+valy['k_y']**2))
        phi = (int(p)-1)*np.pi*0.5
        psi = (int(q)-1)*np.pi*0.5
        if n_0<(beta/k)<n_1:
            solution_xy[(int(p),int(q))] = {'k':k, 'beta':beta,'gamma_x':valx['gamma_x'], 'k_x':valx['k_x'],
                                              'gamma_y':valy['gamma_y'], 'k_y':valy['k_y'],'phi':phi, 'psi':psi}
        
#End timing
end = time.time()

#Print results
print(str(len(solution_xy.keys()))+' modes were found in {} seconds'.format(end-start))
print(pprint.pformat(solution_xy))

16 modes were found in 3.234431743621826 seconds
OrderedDict([((1, 1),
              {'beta': 6475163.9410508843,
               'gamma_x': 2740156.0,
               'gamma_y': 2718140.0,
               'k': 4053667.940115862,
               'k_x': 96371.519936443627,
               'k_y': 359801.77124475129,
               'phi': 0.0,
               'psi': 0.0}),
             ((1, 2),
              {'beta': 6445165.6187854949,
               'gamma_x': 2740156.0,
               'gamma_y': 2645883.0,
               'k': 4053667.940115862,
               'k_x': 96371.519936443627,
               'k_y': 719058.77680608304,
               'phi': 0.0,
               'psi': 1.5707963267948966}),
             ((1, 3),
              {'beta': 6395070.4457954662,
               'gamma_x': 2740156.0,
               'gamma_y': 2521401.0,
               'k': 4053667.940115862,
               'k_x': 96371.519936443627,
               'k_y': 1077162.6485307873,
               'phi': 0.0,
           

In [10]:
solution_xy[1,1]['beta']/solution_xy[1,1]['k']

1.5949936219818039

In [7]:
#Calculating the electric fields E_x, H_y is dominant
#Additional Inputs

region_1 = OrderedDict()
region_2 = OrderedDict()
region_3 = OrderedDict()

for p_q,values in solution_xy.items():
    k = values['k']
    k_x = values['k_x']
    gamma_x = values['gamma_x']
    k_y = values['k_y']
    gamma_y = values['gamma_y']
    phi = values['phi']
    psi = values['psi']
    beta = values['beta']
    #For mode p and distance a
    v = np.sqrt(k**2 * a**2 * (n_1**2 - n_0**2))
    b_find = np.linspace(0,0.999999999,1000000)
    b = b_find[np.argmin(np.absolute(v*np.sqrt(1-b_find)-(p_q[0]-1)*0.5*np.pi-np.arctan(np.sqrt(b_find/(1-b_find)))))]
    u = v * np.sqrt(1 - b)
    w = v * np.sqrt(b)
    #Note w_prime is equal to w because n_s = n_0
    A = np.sqrt((2 * w_ang * mu_0 * P) / (beta * a * (1 + (1/w))))
    C = (beta * a * A**2) / (2 * w_ang * mu_0)  #Constant in power equation for substrate/cladding
    P_core = C * (1 + (((np.sin(u+phi))**2)/(2*w))+(((np.sin(u-phi))**2)/(2*w)))
    P_clad1 = C * (((np.cos(u+phi))**2)/(2*w))
    P_clad2 = C * (((np.cos(u-phi))**2)/(2*w))
    
    
    Confinement = P_core/P
    
    #Region 1
    x = np.linspace(-a,a,200)
    y = np.linspace(-d,d,200)
    H_y1 = A * np.cos(k_x*x-phi) * np.cos(k_y*y-psi) #Hy in region 1
    dH_y1x = -A * k_x * np.sin(k_x*x-phi) * np.cos(k_y*y-psi) #Partial derivative with respect x
    dH_y1xx = -A * k_x**2 * np.cos(k_x*x-phi) * np.cos(k_y*y-psi) #second order partial derivative with respect x
    dH_y1xy = A * k_x * k_y * np.sin(k_x*x-phi) * np.sin(k_y*y-psi) #Partial der. with respect x and with respect y
    dH_y1y = -A * k_y * np.cos(k_x*x-phi) * np.sin(k_y*y-psi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y1y
    E_x = ((w_ang * mu_0)/beta)*H_y1 + (1/(epsilon_0*w_ang*n_1**2*beta))*dH_y1xx
    E_y = (1/(epsilon_0*w_ang*n_1**2*beta)) * dH_y1xy
    E_z = (-1/(epsilon_0*w_ang*n_1**2)) * dH_y1x
    region_1[p_q] = {'x':x, 'y':y, 'H_x':H_x, 'H_y':H_y1, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z, 
                     'Confinement':Confinement}
    #Region 2
    x = np.linspace(a,2*a,100)
    y = np.linspace(d,2*d,100)
    H_y2 = A * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x-a)) * np.cos(k_y*y-psi) #Hy in region 2
    dH_y2x = -A * gamma_x * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x-a)) * np.cos(k_y*y-psi) #Partial derivative with respect x
    dH_y2xx = A * gamma_x**2 * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x-a)) * np.cos(k_y*y-psi) #second order partial derivative with respect x
    dH_y2xy = A * gamma_x * k_y * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x-a)) * np.sin(k_y*y-psi) #Partial der. with respect x and with respect y
    dH_y2y = -A * k_y * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x-a)) * np.sin(k_y*y-psi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y2y
    E_x = ((w_ang * mu_0)/beta)*H_y2 + (1/(epsilon_0*w_ang*n_0**2*beta))*dH_y2xx
    E_y = (1/(epsilon_0*w_ang*n_0**2*beta)) * dH_y2xy
    E_z = (-1/(epsilon_0*w_ang*n_0**2)) * dH_y2x
    region_2[p_q] = {'x':x, 'y':y, 'H_x':H_x, 'H_y':H_y2, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z}
    
    

In [41]:
region_1[2,2]['Confinement']

0.32936504211095696

In [38]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
plt.vlines(-a*1e6,-40000,50000)
plt.vlines(a*1e6,-40000,50000)
plt.plot(region_1[2,2]['x']*1e6,region_1[1,1]['E_x'], color = 'm', label = 'Mode 1,1')
plt.plot(-region_2[2,2]['x']*1e6,region_2[1,1]['E_x'], color = 'm')
plt.plot(region_2[2,2]['x']*1e6,region_2[1,1]['E_x'], color = 'm')
plt.plot(region_1[2,2]['x']*1e6,region_1[1,2]['E_x'], color = 'b', label = 'Mode 1,2')
plt.plot(region_2[2,2]['x']*1e6,region_2[1,2]['E_x'], color = 'b')
plt.plot(-region_2[2,2]['x']*1e6,-region_2[1,2]['E_x'], color = 'b')
plt.plot(region_1[2,2]['x']*1e6,region_1[2,1]['E_x'], color = 'g', label = 'Mode 2,1')
plt.plot(-region_2[2,2]['x']*1e6,-region_2[2,1]['E_x'], color = 'g')
plt.plot(region_2[2,2]['x']*1e6,region_2[2,1]['E_x'], color = 'g')
plt.plot(region_1[2,2]['x']*1e6,region_1[2,2]['E_x'], color = 'k', label = 'Mode 2,2')
plt.plot(-region_2[2,2]['x']*1e6,region_2[2,2]['E_x'], color = 'k')
plt.plot(region_2[2,2]['x']*1e6,region_2[2,2]['E_x'], color = 'k')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc = 0)
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
x = region_1[2,2]['x'] *1e6
y = region_1[2,2]['y'] *1e6
#z = np.resize(region_1[2,1]['E_x'],(200,200))
#z = np.resize(region_1[1,1]['E_x'],(200,1)) @ np.resize(region_1[1,2]['H_y'],(1,200))
z = np.resize(region_1[1,1]['E_x'],(200,1)) @ np.resize(region_1[1,1]['E_x'],(1,200))
fig, ax = plt.subplots()

# filled contours
im = ax.contourf(x, y, z, 100)
plt.xlabel('x')
plt.ylabel('y')
# contour lines
im2 = ax.contour(x, y, z, colors='k')

fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
#Magnetic and electric fields:
    #Region 1
    x = np.linspace(-a,a,200)
    y = np.linspace(-d,d,200)
    H_y1 = A * np.cos(k_x*x_1-phi) * np.cos(k_y*y_1-psi) #Hy in region 1
    dH_y1x = -A * k_x * np.sin(k_x*x_1-phi) * np.cos(k_y*y_1-psi) #Partial derivative with respect x
    dH_y1xx = -A * k_x**2 * np.cos(k_x*x_1-phi) * np.cos(k_y*y_1-psi) #second order partial derivative with respect x
    dH_y1xy = A * k_x * k_y * np.sin(k_x*x_1-phi) * np.sin(k_y*y_1-psi) #Partial der. with respect x and with respect y
    dH_y1y = -A * k_y * np.cos(k_x*x_1-phi) * np.sin(k_y*y_1-psi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y1y
    E_x = ((w_ang * mu_0)/beta)*H_y1 + (1/(epsilon_0*w_ang*n_1**2*beta))*dH_y1xx
    E_y = (1/(epsilon_0*w_ang*n_1**2*beta)) * dH_y1xy
    E_z = (-1/(epsilon_0*w_ang*n_1**2)) * dH_y1x
    region_1[p_q] = {'x':x, 'y_1':y_1, 'H_x':H_x, 'H_y':H_y1, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z}

    #Region 2
    x = np.linspace(a,2*a,100)
    y = np.linspace(d,2*d,100)
    H_y2 = A * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.cos(k_y*y_2-psi) #Hy in region 2
    dH_y2x = -A * gamma_x * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.cos(k_y*y_2-psi) #Partial derivative with respect x
    dH_y2xx = -A * gamma_x**2 * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.cos(k_y*y_2-psi) #second order partial derivative with respect x
    dH_y2xy = A * gamma_x * k_y * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.sin(k_y*y_2-psi) #Partial der. with respect x and with respect y
    dH_y2y = -A * k_y * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.sin(k_y*y_2-psi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y2y
    E_x = ((w_ang * mu_0)/beta)*H_y2 + (1/(epsilon_0*w_ang*n_0**2*beta))*dH_y2xx
    E_y = (1/(epsilon_0*w_ang*n_0**2*beta)) * dH_y2xy
    E_z = (-1/(epsilon_0*w_ang*n_0**2)) * dH_y2x
    region_2[p_q] = {'x':x, 'y':y, 'H_x':H_x, 'H_y':H_y2, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z}

    #Region 3
    x = np.linspace(a,2*a,100)
    y = np.linspace(d,2*d,100)
    H_y3 = A * np.cos(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-psi) #Hy in region 3
    dH_y3x = -A * k_x * np.sin(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-psi)  #Partial derivative with respect x
    dH_y3xx = -A * k_x**2 * np.cos(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-phi) #second order partial derivative with respect x
    dH_y3xy = A * k_x * gamma_y * np.sin(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-psi) #Partial der. with respect x and with respect y
    dH_y3y = -A * gamma_y * np.cos(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-phi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y3y
    E_x = ((w_ang * mu_0)/beta)*H_y3 + (1/(epsilon_0*w_ang*n_0**2*beta))*dH_y3xx
    E_y = (1/(epsilon_0*w_ang*n_0**2*beta)) * dH_y3xy
    E_z = (-1/(epsilon_0*w_ang*n_0**2)) * dH_y3x
    region_3[p_q] = {'x':x, 'y':y, 'H_x':H_x, 'H_y':H_y3, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z}

    